In [1]:
# imports
%pip install -r bert_race/requirements.txt
%pip install transformers==4.2.2

!pip install tasks
!pip install pytorch_pretrained_bert==0.4.0 
!pip install seqeval

%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tasks-2.8.0-py3-none-any.whl (12 kB)
  Using cached datadispatch-1.0.0-py3-none-any.whl (2.6 kB)
  Using cached pytorch_pretrained_bert-0.4.0-py3-none-any.whl (45 kB)
  Using cached seqeval-1.2.2-py3-none-any.whl


In [3]:
from collections import defaultdict
import numpy as np
d = defaultdict(0)

TypeError: first argument must be callable or None

In [4]:
d = {
    
}

In [7]:
x = d.get('best_acc', np.inf)
d['best_acc'] = x
d

{'best_acc': inf}

In [2]:
import transformers 
transformers.__version__

'4.2.2'

In [3]:
from utils.tokenization_utils import read_data

In [4]:
bucket = 'support-bert-data'
data_location = f's3://{bucket}'

In [5]:
train_examples = read_data(data_location,'train')
dev_examples = read_data(data_location,'dev')
test_examples = read_data(data_location,'test')

After sorted: support-bert-data/train/0.txt
After sorted: support-bert-data/dev/0.txt
After sorted: support-bert-data/test/0.txt


In [6]:
# %cp bert_race/pytorch_pretrained_bert/tokenization.py utils/tokenization.py
# %cp bert_race/pytorch_pretrained_bert/file_utils.py utils/file_utils.py

from utils.tokenization import BertTokenizer
from utils.tokenization_utils import convert_examples_to_features

In [7]:
bert_model = "bert-base-uncased"
do_lower_case = True
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=do_lower_case)

In [21]:
max_seq_length=128 # 256 doesn't work, I don't know why.
train_features = convert_examples_to_features(
            train_examples, tokenizer, max_seq_length, True)
test_features = convert_examples_to_features(
            test_examples, tokenizer, max_seq_length, True)
dev_features = convert_examples_to_features(
            dev_examples, tokenizer, max_seq_length, True)

3289 race_id: support-bert-data/train/0:0, context_sentence: when tech opens info session and when he tries to close it, it hangs and will not close. i have included a screen shot as to what process  hangs in the device manager. i have tried to uninstall and reinstall the pdf application but it did not help, start_ending: what kind of issue occurred?, ending_0: a performance issue., ending_1: [NOA], ending_2: a protocol issue., label: 0
0
1025 race_id: support-bert-data/test/0:0, context_sentence: [AN56] data is showing as queued, almost all to [AN16], [AN31] seems to transmit okay. please see screen shot and [AN25] requirements log. i also submitted error report from admin client on device., start_ending: how can we help you?, ending_0: [NOA], ending_1: i have a question or i would like to know how to perform a specific action or task related to the usage of {[AN25]}., ending_2: i have an issue when using {[AN25]} and i need support., label: 2
0
557 race_id: support-bert-data/dev/0:0,

In [22]:
import os

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%cp bert_race/pytorch_pretrained_bert/modeling.py utils/modeling.py
%cp bert_race/pytorch_pretrained_bert/optimization.py utils/optimization.py

import torch
from utils.modeling import BertForMultipleChoice, BertConfig
from utils.file_utils import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME
from utils.optimization import BertAdam, WarmupLinearSchedule

In [23]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset, IterableDataset)
from tqdm import tqdm, trange

In [24]:
# from utils.tokenization_utils import build_tensor
def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

def build_tensor(features):
    all_input_ids = torch.tensor(select_field(features, 'input_ids'),
                                 dtype=torch.long)
    all_input_mask = torch.tensor(select_field(features, 'input_mask'),
                                  dtype=torch.long)
    all_segment_ids = torch.tensor(select_field(features, 'segment_ids'),
                                   dtype=torch.long)
    all_label = torch.tensor([f.label for f in features],
                             dtype=torch.long)
    return TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)

train_data = build_tensor(train_features)
test_data = build_tensor(test_features)
dev_data = build_tensor(dev_features)

train_data

In [25]:
# args
seed = 30
do_lower_case = True
bert_model = "bert-base-uncased"
gradient_accumulation_steps=8
train_batch_size=2
num_train_epochs=5
learning_rate=1e-5

In [26]:
import yaml
with open ('test_config.yml', 'r') as fp:
    config = yaml.load(fp, Loader=yaml.FullLoader)

In [27]:
from loss import LOSS_REGISTRY, LossCriterion, CeCriterion, SymKlCriterion
ce_loss = LossCriterion.CeCriterion
adv_loss = LossCriterion.SymKlCriterion

In [28]:
config.update({'adam_eps': 6, 
              'adv_epsilon': 1e-6,
              'adv_train': 1,
              'adv_alpha': 1,
              'adv_k': 1,
              'adv_noise_var': 1e-5,
              'adv_norm_level': 0,
              'adv_p_norm': 'inf',
              'adv_step_size': 1e-3,
              'batch_size': 8,
              'bin_on': False,
              'cuda': 1,
              'encoder_type': None,
              'fp16': True,
              'fp16_opt_level': 'O1',
              'global_grad_clipping': 1.0,
              'grad_accumulation_step': 1,
              'grad_clipping': 0,
              'learning_rate': 5e-5,
              'local_rank': -1,
              'mkd_opt': 0,
              'optimizer': 'adam',
              'scheduler_type': 'ms',
              'task_def_list': None,
              'warmup': 0.1,
              'warmup_schedule': 'warmup_linear',
              'weight_decay': 0,
              'weighted_on': False,
              'state_dict': None})
config.update({'loss': ce_loss, 'adv_loss': adv_loss})

In [ ]:
sweep_config_adv = {'adam_eps': 6, 
              'adv_epsilon': 1e-6,
              'adv_train': 1,
              'adv_noise_var': 1e-5,
              'adv_norm_level': 0,
              'adv_step_size': 1e-3,
              'batch_size': 8,
              'bin_on': False,
              'cuda': 1,
              'encoder_type': None,
              'fp16': True,
              'fp16_opt_level': 'O1',
              'global_grad_clipping': 1.0,
              'grad_accumulation_step': 1,
              'grad_clipping': 0,
              'parameters': {
                  'learning_rate': {
                      'values': [5e-4, 1e-5, 3e-5, 5e-5]
                  },
                  'optimizer': {
                      'values': ['adam', 'radam']
                  },
                  'adv_k': {
                      'values': [1, 3, 5]
                  },
                  'adv_p_norm': {
                      'values': ['inf', 'l1', 'l2']
                  },
                  'batch_size': {
                      'values': [16, 32, 64]
                  },
                  'adv_alpha': {
                      'values': [1, 3, 5]
                  },
                  
                  
              }
              'local_rank': -1,
              'mkd_opt': 0,
              'scheduler_type': 'ms',
              'task_def_list': None,
              'warmup': 0.1,
              'warmup_schedule': 'warmup_linear',
              'weight_decay': 0,
              'weighted_on': False,
              'state_dict': None}

In [29]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

In [30]:
from race_utils.race_mtdnn import race_mtdnn

In [31]:
# tomorrow, work on this error - model = SANBertModel 
# has now to be toasmaster model!
# mtdnn_model = MTDNNModel(config, device='gpu', num_train_step=len(train_dataloader))

In [32]:
config.update({'batch_size': 2}) #trying out for memory
rmtdnn_model = race_mtdnn(config, None, None, len(train_dataloader), seed)

INFO:utils.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:utils.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:utils.modeling:extracting archive file /root/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpr5earn3l
INFO:utils.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_si

TypeError: super(type, obj): obj must be an instance or subtype of type

In [20]:
rmtdnn_model.update(train_dataloader)

Iteration:   0%|          | 0/1645 [00:00<?, ?it/s]

[2021-10-16 12:58:36.371 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-c630bdb4e3ad8d68ab6e5727a214:120 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-16 12:58:36.404 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-c630bdb4e3ad8d68ab6e5727a214:120 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
torch.Size([6, 256, 768])
torch.Size([6, 256, 768])
torch.Size([6, 256, 768])


Iteration:   0%|          | 1/1645 [00:00<22:26,  1.22it/s]


torch.Size([6, 256, 768])
torch.Size([6, 256, 768])
torch.Size([6, 256, 768])


RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 14.76 GiB total capacity; 6.59 GiB already allocated; 14.75 MiB free; 6.80 GiB reserved in total by PyTorch)

In [26]:
rmtdnn_model.adv_teacher
rmtdnn_model.adv_task_loss_criterion

[SymKlCriterion()]

In [ ]:
LossCriterion.CeCriterion

In [ ]:
for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(rmtdnn_model.device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        logits = rmtdnn_model.mnetwork(input_ids, segment_ids, input_mask)

In [ ]:
for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(rmtdnn_model.device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        break

In [ ]:
rmtdnn_model.mnetwork(input_ids, segment_ids, input_mask, label_ids)

In [ ]:
adv_loss

In [ ]:
snlisample:
  data_format: PremiseAndOneHypothesis
  enable_san: true
  labels:
  - contradiction
  - neutral
  - entailment
  metric_meta:
  - ACC
  loss: CeCriterion
  kd_loss: MseCriterion
  adv_loss: SymKlCriterion
  n_class: 3
  task_type: Classification